## 데이타 증강 - 일반대화는 증강하지 않는다.

In [1]:
import os
import pandas as pd
import tensorflow
import numpy as np

In [2]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '../../data', 'raw_csv')
train_data_path = os.path.join(data_dir, 'merged_train_train.csv')

train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,673,직장 내 괴롭힘 대화,김과장 라인 잘 타. 누가 회사 이끄는 건데.\n 아무래도 이건 부당한 것 같습니다...
1,243,협박 대화,이게 어떻게 된거야.? 엄마 수술은 어떻게 된거냐고\n 엄마가 이제 괜찮다고 해서 ...
2,2006,갈취 대화,이사님 이 사진. 모르지 않으실텐데요?\n이. 이건. \n이사님이 여기까지 오신거 ...
3,1073,협박 대화,너 지금 나랑 헤어지자고 한거야? 죽고싶어?\n 그만하자\n 너 내가 헤어지자고 말...
4,893,기타 괴롭힘 대화,철수 씨는 그 학교 나와서 그 연봉 받고 살만해요?\n그게 무슨 말씀이시죠?\n아니...


In [3]:
print('전체 샘플수 :', (len(train_data)))
train_data.info()

전체 샘플수 : 3709
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3709 entries, 0 to 3708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3709 non-null   int64 
 1   class         3709 non-null   object
 2   conversation  3709 non-null   object
dtypes: int64(1), object(2)
memory usage: 87.1+ KB


**한국어 불용어 사전**

In [4]:
# 불용어 제거
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ?.!,]+", " ", sentence)
    
    sentence = sentence.strip()

    return sentence

In [5]:
import re

def preprocess_korean(sentence):
    # 한글 외 모든 문자 제거
    sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', sentence)
    # 연속된 공백 제거
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence


**동의어**

In [6]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)

# 전체': ['전체', '전적', '전부', '총체', '전반']
# '인': ['인', '인간', '인물', '사람', '인류']
#print(wordnet)

In [7]:
# wordnet에서 동의어 리스트를 가져온다. 
# 동의어가 없을 수도 있다
def synonyms_from_wordnet(word):
    synonyms = []
    try:
        for list in wordnet[word] :
             synonyms.append(list)
    except KeyError:
        pass
    
    return synonyms

In [8]:
#'전체': ['전체', '전적', '전부', '총체', '전반']
synonyms = synonyms_from_wordnet( "전체" )
for word in synonyms : 
    print(word)

전체
전적
전부
총체
전반


In [9]:
# word list로 구성된 sentense를 동의어로 바꿔준다
def replace_to_synonyms(input_words, count): 
     
    # new_words
    new_words = []
    
    #
    word_list = list(set([word for word in input_words]))
    random.shuffle(word_list)
    
    replace_count = 0
    for word in word_list : 
        # 동의어 리스트를 가져온다. 
        synonyms = synonyms_from_wordnet( word )
        
        if len(synonyms) == 0 : continue
        
        synonym = random.choice(list(synonyms))
        if( word == synonym) : continue
            
        new_words = [synonym if wd == word else wd for wd in input_words]
        
        replace_count +=1
        
        if replace_count >= count : break
    
    # 
    #if len(new_words) > 0 : 
    #    sentence = ' '.join(new_words)
    #   new_words = sentence.split(" ")
    
       
    return new_words
               

In [10]:
def gen_sentence_by_swap_word(inputs_words, count):
    
    # new_words
    new_words = []
    
    #
#    word_list = list(set([word for word in input_words]))
#    random.shuffle(word_list)
    
    return new_words

In [11]:
def DataAugmentation(sentense, alpha_sr=0.2, aug_count=1):
    
    augmented_sentences = []
    
    # 한글만 처리
    sentense = preprocess_korean(sentense)
    sentense = sentense.split(' ')
    words = [word for word in sentense if word != ""]
    num_words = len(words)
       
    # 동의어로 단어 바꿔 문장 생성하기
    n_sr = max(1, int(alpha_sr*num_words))
    
    for _ in range(aug_count):
        a_words = replace_to_synonyms( words, n_sr )
          
        if( len(a_words) > 0 ):
            augmented_sentences.append(" ".join(a_words))
     
    
    
    # 생성된 문장들 shuffle
    random.shuffle(augmented_sentences)
    return augmented_sentences

In [13]:
new_class_list = []
new_conversation_list = []
new_idx_list = []

idx = len(train_data['conversation']);

filtered_data = train_data[train_data['class'] != '일반 대화']
filtered_data = filtered_data.reset_index(drop=True)

         
for i in range(len(filtered_data['conversation'])):

    class_ = filtered_data['class'][i]
    
    num_aug = 3
    
    augmented_sentences = DataAugmentation(train_data['conversation'][i], aug_count=num_aug)
    
    if not augmented_sentences : # 리스트가 비어 있으면 True
        continue
        
    for j in range(min(num_aug, len(augmented_sentences))):
        #if not augmented_sentences[j] : continue;
        new_class_list.append( class_ )
        new_conversation_list.append(augmented_sentences[j])
        idx =idx+1
        new_idx_list.append(int(idx))

print(len(new_class_list))    
print(len(new_conversation_list)) 
        

6357
6357


In [14]:
filtered_data['conversation'][0:1]

0    김과장 라인 잘 타. 누가 회사 이끄는 건데.\n 아무래도 이건 부당한 것 같습니다...
Name: conversation, dtype: object

In [15]:
new_conversation_list[0:3]

['김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장이 그렇다하면 어쩔 건데 그러니까 네가 아직도 과장인 거야 저는 하늘 부끄럼 없습니다 요즘 같은 시대에 아직도 이런 사람이 있네 네 알아서 해라 너같은 답답이는 오히려 방해돼 부장님이 그러시면 저는 사나이 신고 센터에 갈 수밖에 없습니다 자네 지금 나 협박하는 거야 자신 있어 두고봐 아니에요 부장님 제가 뭘 하면 되나요 잘 좀 하자 이번 건 안 따오면 아니면 다음 승진도 물 건너 간거다 물론 넌 못할 가능성 높겠지만 네 알겠습니다',
 '김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장이 그렇다하면 어쩔 건데 그러니까 네가 아직도 과장인 거야 저는 하늘 부끄럼 없습니다 요즘 같은 시대에 아직도 이런 사람이 있네 네 알아서 해라 너같은 답답이는 오히려 방해돼 부장님이 그러시면 저는 사내 신고 센터에 갤런 수밖에 없습니다 자네 지금 나 협박하는 거야 자신 있어 두고봐 아니에요 부장님 제가 뭘 하면 되나요 잘 좀 하자 이번 건 안 따오면 아니면 다음 승진도 물 건너 간거다 물론 넌 못할 가능성 높겠지만 네 알겠습니다',
 '김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장이 그렇다하면 어쩔 건데 그러니까 네가 아직도 과장인 거야 저는 하늘 부끄럼 없습니다 요즘 같은 시대에 아직도 이런 사람이 있네 네 알아서 해라 너같은 답답이는 오히려 방해돼 부장님이 그러시면 저는 사내 신고 센터에 갈 수밖에 없습니다 자네 지금 나 협박하는 거야 자신 있어 두고봐 아니에요 부장님 제가 뭘 하면 되나요 잘 좀 하자 이번 건 안쪽 따오면 아니면 다음 승진도 물 건너 간거다 물론 넌 못할 가능성 높겠지만 네 알겠습니다']

In [16]:
aug_data = pd.DataFrame([new_idx_list, new_class_list, new_conversation_list],
                  index=['idx','class','conversation'],
                 columns=[x for x in range(len(new_class_list))])
aug_data  = aug_data .transpose()

In [17]:
aug_data.head()

,idx,class,conversation
0,3711,직장 내 괴롭힘 대화,김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장...
1,3712,직장 내 괴롭힘 대화,김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장...
2,3713,직장 내 괴롭힘 대화,김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장...
3,3714,협박 대화,이게 어떻게 된거야 엄마 수술은 어떻게 된거냐고 엄마가 이제 괜찮다고 해서 딱 봐도...
4,3715,협박 대화,이게 어떻게 된거야 엄마 수술은 어떻게 된거냐고 엄마가 이제 괜찮다고 해서 딱 봐도...


In [18]:
aug_train_data = aug_data.sample(frac=1).reset_index(drop=True)

aug_train_data['idx'] = [(len(train_data['conversation'])+x+1)for x in range(len(aug_train_data))]
aug_train_data = aug_train_data.reset_index(drop=True)

In [19]:
aug_train_data.head()

,idx,class,conversation
0,3710,기타 괴롭힘 대화,돈 있는 것좀 다 줘봐 이게 다에요 오천원 거짓말하지마 거짓말아니에요 이게 다에요 ...
1,3711,협박 대화,저기요 이거 영상으로 왜 올리셨어요 사전에 동의하에 목소리 나간다고 말씀드렸습니다 ...
2,3712,직장 내 괴롭힘 대화,야 너 선배를 봤으면 인사를 해야지 뭐해 앗 선배 안녕하세요 됐고 폰 좀 줘봐 네 ...
3,3713,협박 대화,담배 좀 빌립시다 학생들같은데 그냥 가라 거 같이 좀 핍시다 학생이 무슨 담배야 거...
4,3714,협박 대화,자위 영상 보내 빨리 아 그리고 궁금한게 있는데 학교 어디 다녀 응 여기있어 그건 ...


In [20]:
aug_train_data.tail()

,idx,class,conversation
6352,10062,기타 괴롭힘 대화,야 이 책 내가 좀 빌려간다 응 너 저번에 빌려간 내 수학책도 안줬잖아 아 맞다 이...
6353,10063,직장 내 괴롭힘 대화,이번 방학 과정 있어 친구들이랑 제주도 여행 가려고 몇 박 몇 일로 과정 중이야 박...
6354,10064,협박 대화,감독님 저 더 이상 시나리오 못짜겠습니다 대신 그동안 작업해놨는 것을 이어서 다른 ...
6355,10065,협박 대화,실례합니다 이 자리 맥주 있나요 네 저한테 물으시는 건가요 네 이 좌석이 맥주 있는...
6356,10066,기타 괴롭힘 대화,어이 부딪혔으면 정중하게 사과를 해야할 거 아이가 죄송합니다 임마 말하는 거 보니깐...


In [21]:
print(len(aug_train_data))

6357


In [22]:
class_counts = aug_train_data['class'].value_counts()

print(class_counts)

기타 괴롭힘 대화      1659
갈취 대화          1616
직장 내 괴롭힘 대화    1561
협박 대화          1521
Name: class, dtype: int64


In [23]:
aug_train_data.to_csv('../../data/aug_csv/aug_synonym_train_only.csv', index=False, encoding='utf-8-sig')  # index=False는 인덱스를 저장하지 않도록 함

In [26]:
aug_data_path ="../../data/aug_csv/aug_synonym_train_only.csv"
aug_data = pd.read_csv(aug_data_path)
aug_data.head()

,idx,class,conversation
0,3710,기타 괴롭힘 대화,돈 있는 것좀 다 줘봐 이게 다에요 오천원 거짓말하지마 거짓말아니에요 이게 다에요 ...
1,3711,협박 대화,저기요 이거 영상으로 왜 올리셨어요 사전에 동의하에 목소리 나간다고 말씀드렸습니다 ...
2,3712,직장 내 괴롭힘 대화,야 너 선배를 봤으면 인사를 해야지 뭐해 앗 선배 안녕하세요 됐고 폰 좀 줘봐 네 ...
3,3713,협박 대화,담배 좀 빌립시다 학생들같은데 그냥 가라 거 같이 좀 핍시다 학생이 무슨 담배야 거...
4,3714,협박 대화,자위 영상 보내 빨리 아 그리고 궁금한게 있는데 학교 어디 다녀 응 여기있어 그건 ...
